In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from random import randint
import socket
import socks

In [ ]:
def get_job_title(soup): 
    jobs = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            jobs.append(a["title"])
    return(jobs)

def get_company(soup): 
    companies = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        company = div.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
    else:
        sec_try = div.find_all(name="span", attrs={"class":"result-link-source"})
        for span in sec_try:
            companies.append(span.text.strip())
    return(companies)

def get_location(soup): 
    locations = []
    spans = soup.findAll("span", attrs={"class": "location"})
    for span in spans:
        locations.append(span.text)
    return(locations)

def get_posting_date(soup):
    dates = []
    for i in soup.findAll("span", attrs={"class": "date"}):
        dates.append(i.text)
    return dates

def get_rating(soup):
    tag = soup.findAll("meta", attrs={"itemprop": "ratingValue"})
    if len(tag) == 0: 
        rating = 0
    else:
        rating = tag[0].get('content')
    return rating

def get_links(soup):
    links = []
    for i in soup.find_all(name="a", attrs={"target":"_blank"}):
        href = i.get('href')
        if '/rc/clk' in href:
            links.append(href[7:])
    return links

def get_soup(url):
    page = requests.get(url)
    if page.status_code == 200:
        soup = BeautifulSoup(page.text, "html.parser")
    else:
        print('Status code is not 200')
        soup = 0
        time.sleep(120)
    return soup

def get_job_descr(soup):
    descr = ''

        
    for i in soup.find_all(name='ul'):
        descr += ' ' + (str(i.text))
    descr = descr.replace('\n',' ')
    return descr

def get_other_text(soup):
    text = ''
    for i in soup.find_all(name='p'):
        text += ' ' + (str(i.text))

    for i in soup.find_all(name='li'):
        text += ' ' + (str(i.text))
    
    for i in soup.find_all(name='br'):
        text += ' ' + (str(i.text))
        
    text = text.replace('\n',' ')
    return text

In [ ]:
# url = "https://www.indeed.com/jobs?q=data+scientist&sort=date"
# cities = {'New+York%2C+NY':320,'San+Francisco%2C+CA':380,'Washington%2C+DC':540,'Texas':220,'Seattle%2C+WA':300,
#          'Los+Angeles%2C+CA':120,'Chicago%2C+IL':120}

cities = {'https://www.indeed.com/jobs?q=Data+Scientist&sort=date&start=' : 10,
          'https://www.indeed.co.uk/jobs?q=data+scientist&l=London&sort=date&start=' : 0}


# socks.set_default_proxy(socks.SOCKS5,'localhost',9150)
# socket.socket=socks.socksocket

data = pd.DataFrame(columns=['title', 'posting_date', 'company_rating', 
                             'company', 'location', 'description', 'text', 'url'])

for c in cities:
    for count in range(0,cities[c]+1,10):
#         url = f'https://www.indeed.com/jobs?q=data+scientist&l={c}&sort=date&start={count}'
        url = c+str(cities[c])
        soup = get_soup(url)
        if soup ==0:
            continue
        links = get_links(soup)
        df = pd.DataFrame(columns=data.columns)
        df['title'] = get_job_title(soup)
        df['company'] = get_company(soup)
        df['location'] = get_location(soup)
        df['posting_date'] = get_posting_date(soup)
        for i,l in enumerate(links):
            link = 'https://www.indeed.com/viewjob' +l
            df.loc[i]['url'] = link
            print(link)
            page = get_soup(link)
            if soup == 0:
                continue
            df.loc[i]['description'] = get_job_descr(page)
            df.loc[i]['text'] = get_other_text(page)
            df.loc[i]['company_rating'] = get_rating(page)
            time.sleep(randint(3,5))
        data = pd.concat([data,df],axis=0)

In [ ]:
data.shape

In [ ]:
def clear(x):
    trash = ' Hiring LabCareer AdviceBrowse JobsBrowse CompaniesSalariesFind CertificationsEmployer EventsWork at IndeedCountriesAboutHelp Center Do Not Sell My Personal InformationPrivacy Center'
    if trash in x:
        x.replace(trash,'')
    return x

data.dropna(inplace=True)
data.drop_duplicates(inplace=True)
data['description'] = data.description.map(lambda x: clear(x))
data.reset_index(inplace=True, drop = True)

In [ ]:
data = data[(data.posting_date=='1 day ago')]

In [ ]:
data.to_csv('new.csv',index=False)